# Learning-to-rank : pyTerrier - OpenNIR

Dans cette partie, on s'intéresse à l'utilisation de modèles neuronaux pour la recherche d'information. 
Les modèles neuronaux utilisés ont été rassemblés dans la librairie [OpenNIR](https://opennir.net/). 
On explorera également le modèle T5 avec le plugin [monoT5](https://github.com/terrierteam/pyterrier_t5).

In [ ]:
!pip install --upgrade python-terrier
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.7 MB/s eta 0:00:00
  Preparing me

## Initialisation
De façon similaire au TP1, on initialise PyTerrier. Nous allons travailler sur le dataset CORD19. le bloc suivant est une répétition du code en TP1.

In [ ]:
import pyterrier as pt
if not pt.started():
    pt.init(tqdm='notebook')
import onir_pt

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [ ]:
import os

dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = dataset.get_topics(variant='description')
qrels = dataset.get_qrels()

indexer = pt.index.IterDictIndexer('./cord19-index')
indexref = indexer.index(dataset.get_corpus_iter(), fields=('title', 'abstract'))
index = pt.IndexFactory.of(indexref)

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: s] [458B] [328kB/s]
[WARNING] Download failed: Expected md5 hash to be 0307a37b6b9f1a5f233340a769d538ea but got 213064f8c4c4654f4e2d7c157b2e1fce
[INFO] [starting] https://mirror.ir-datasets.com/0307a37b6b9f1a5f233340a769d538ea
[INFO] [finished] https://mirror.ir-datasets.com/0307a37b6b9f1a5f233340a769d538ea: s] [18.7kB] [8.23MB/s]
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: s] [458B] [324kB/s]
[WARNING] Download failed: Expected md5 hash to be 8138424a59daea0aba751c8a891e5f54 but got 213064f8c4c4654f4e2d7c157b2e1fce
[INFO] [starting] https://mirror.ir-datasets.com/8138424a59daea0aba751c8a891e5f54
[INFO] [finished] https://mirror.ir-datasets.com/8138424a59daea0aba751c8a891e5f54: s] [1.14MB] [57.9MB/s]
[INFO] [starting] building d

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

19:32:53.792 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (8is9x9sc) - further warnings are suppressed
19:34:02.767 [ForkJoinPool-1-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.indexDocuments(BasicIndexer.java:270)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Inde

In [ ]:
print(os.path.exists("terrier_index.zip"))

False


In [ ]:
#Si le chargement de la collection est trop long à cause du débit ou autres raisons,
# il est possible de récupérer directement l'index fourni par Terrier
import os

if not os.path.exists("terrier_index.zip"):
  !wget http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/terrier_index.zip
  !unzip -j terrier_index.zip -d terrier_index

index_ref = pt.IndexRef.of("./terrier_index/data.properties")
index = pt.IndexFactory.of(index_ref)

--2023-06-11 19:34:13--  http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/terrier_index.zip
Resolving www.dcs.gla.ac.uk (www.dcs.gla.ac.uk)... 130.209.240.1
Connecting to www.dcs.gla.ac.uk (www.dcs.gla.ac.uk)|130.209.240.1|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/terrier_index.zip [following]
--2023-06-11 19:34:14--  https://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/terrier_index.zip
Connecting to www.dcs.gla.ac.uk (www.dcs.gla.ac.uk)|130.209.240.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42017186 (40M) [application/zip]
Saving to: ‘terrier_index.zip’

terrier_index.zip   100%[===================>]  40.07M  9.09MB/s    in 4.4s    

2023-06-11 19:34:19 (9.09 MB/s) - ‘terrier_index.zip’ saved [42017186/42017186]

Archive:  terrier_index.zip
  inflating: terrier_index/data.lexicon.fsomapfile  
  inflating: terrier_index/data.properties  
  inflating: terrier_index/

## Modèles de ré-ordonancement neuronaux "from scratch"

Les modèles de ré-ordonnancement dans OpenNIR sont constitués de deux éléments :
*  ranker: un modèle d'ordonnancement (e.g., drmm, knrm, pacrr, ...). Cf la liste des [Rankers](https://opennir.net/rankers.html). Il est également possible de rajouter des modèles d'ordonnancement en étendant la classe Ranker. 
*  vocab : défnit comment le texte est encodé par le modèle (e.g., wordvec_hash, bert, ...). Cela permet ainsi de tester plusieurs méthodes de représentation. Plus de détails concernant le [vocab](https://opennir.net/vocab.html).

Les modèles de réordonnancement s'appuient sur une première étape d'ordonnancement (souvent BM25), récupèrent ensuite les textes des top documents et appliquent ensuite le modèle neuronal.



In [ ]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')


config file not found: config
[2023-06-11 19:34:20,775][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip: 98.4%| 671M/682M [30.45s<469ms, 23.6MB/s]

Une fois le modèle chargé, il est nécessaire de mettre en place la pipeline d'évaluation vue dans le tp1. 
Le modèle neuronal n'est pas efficace car il n'a pas été entraîné et utilise des poids aléatoires.

In [ ]:
br = pt.BatchRetrieve(index) % 100
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

## Entraînement des modèles sur les jeux de données

Pour entraîner les modèles, il est nécessaire de construire la pipeline de modèles/transformations à utliser et ensuite d'appliquer la fonction .fit() à la pipeline. 
Le code ci-dessous prend beaucoup de temps. Il est donné à titre indicatif si vous souhaitez l'utiliser sur des serveurs adaptés. L'étape suivante permet de charger directement les poids du modèle pré-entraîné à l'avance et mis à disposition de la communauté.

Dans ce qui suit, on utilise le jeu de données MS MARCO medical pour pré-entraîner le modèle qui sera ensuite appliqué sur CORD19. 
Il est également possible de garder seulement le jeu de données CORD19, de le découper en train/val/test et regarder les performances. 

In [ ]:
# Apprentissage du modèle sur des données médicales (MS MARCO medical)
from sklearn.model_selection import train_test_split
train_ds = pt.datasets.get_dataset('irds:msmarco-passage/train/medical')
train_topics, valid_topics = train_test_split(train_ds.get_topics(), test_size=50, random_state=42)

# Indexation de MS MARCO pour la première étape d'ordonnancement et récupérer les textes (pour le ranker openNIR)
indexer = pt.index.IterDictIndexer('./terrier_msmarco-passage')
tr_index_ref = indexer.index(train_ds.get_corpus_iter(), fields=('text',), meta=('docno',))

pipeline = (pt.BatchRetrieve(tr_index_ref) % 100 # récupère les 100 premiers documents
            >> pt.text.get_text(train_ds, 'text') # récupère le texte de ces documents
            >> pt.apply.generic(lambda df: df.rename(columns={'text': 'abstract'})) # renomme la colonne
            >> knrm) # applique le re-ranker


pipeline.fit(
    train_topics,
    train_ds.get_qrels(),
    valid_topics,
    train_ds.get_qrels())

Pour éviter l'étape d'entraînement ici, on utlise une version pré-entraînée.

**Important** : penser à supprimer la mémoire des re-rankers qui vont être mis à jour avec les modèles pré-entrainés.

In [ ]:
del knrm # free up the memory before loading a new version of the ranker
knrm = onir_pt.reranker.from_checkpoint('https://macavaney.us/knrm.medmarco.tar.gz', text_field='abstract', expected_md5="d70b1d4f899690dae51161537e69ed5a")

Il est ensuire possible de lancer la pipeline d'évaluation avec ce nouveau modèle. Les résultats sont meilleurs !

In [ ]:
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    baseline=0,       ## spécifie quelle est le modèle de référence pour calculer les améliorations.
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

**Exercice 1** 
Autre pipeline. A vous de deviner ce qu'elle fait !

In [ ]:
cutoffs = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
dph = pt.BatchRetrieve(index)
res = pt.Experiment(
    [dph % cutoff >> pt.text.get_text(dataset, 'abstract') >> knrm for cutoff in cutoffs],
    dataset.get_topics('description'),
    dataset.get_qrels(),
    names=[f'c={cutoff}' for cutoff in cutoffs],
    eval_metrics=["map", "recip_rank", "ndcg", "ndcg_cut.10", "mrt"]
)
res

## Modèle Vanilla BERT

**Exercice 2**

Sur le même principe que le modèle KNRM, analyser les performances du modèle vanilla BERT sans et avec pré-entraînement.
Pour la version du modèle pré-entraîné, on utilisera le checkpoint ['https://macavaney.us/scibert-medmarco.tar.gz']('https://macavaney.us/scibert-medmarco.tar.gz') avec le paramètre expected_md5="854966d0b61543ffffa44cea627ab63b".

Synthétisez toutes les mesures d'évaluation dans un même tableau (bm25, knrm et Vanilla Bert / avec/sans entraînement).

# Autres modèles - hors OpenNIR
PyTerrier a mis a disposition l'implémentation d'autres modèles neuronaux:
- ColBERT : https://github.com/terrierteam/pyterrier_colbert
- T5 : https://github.com/terrierteam/pyterrier_t5
- Doc2Query : https://github.com/terrierteam/pyterrier_doc2query
- DeepCT : https://github.com/terrierteam/pyterrier_deepct
- ANCE : https://github.com/terrierteam/pyterrier_ance

Un exemple de code de code est donnée ci-dessous avec Mono-T5 :

In [ ]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker(text_field='abstract')

br = pt.BatchRetrieve(index) % 30
pipeline = (br >> pt.text.get_text(dataset, 'abstract') >> monoT5)
pt.Experiment(
    [br, pipeline],
    dataset.get_topics('description'),
    dataset.get_qrels(),
    names=['DPH', 'DPH >> T5'],
    eval_metrics=["map", "recip_rank", "ndcg", "ndcg_cut.10", "mrt"]
)